In [5]:
import os
import shutil
from pathlib import Path

def decimal_to_base26(n):
    """将十进制数字转换为26进制字母表示"""
    result = ""
    while n > 0:
        n, remainder = divmod(n - 1, 26)
        result = chr(remainder + ord('a')) + result
    return result

def move_and_rename_images(directory):
    """
    递归遍历指定目录，将所有图片文件移动到该目录并重命名。
    完成后，根据成功处理的文件记录，删除非图片文件和未成功处理的图片及空目录。
    同时记录操作到操作日志文件。
    
    图片文件被重命名为基于26进制的字母组合，如'a', 'b', ..., 'z', 'aa', 'ab', ...
    
    参数:
    directory (str or Path): 目标目录路径
    """
    counter = 0
    image_extensions = ['.jpg', '.jpeg', '.png', '.gif', '.bmp']
    successful_files = set()
    operation_log_path = Path(directory) / "operation_log.txt"

    target_dir = Path(directory)
    if not target_dir.is_dir():
        print(f"目录不存在: {directory}")
        return

    failed_counter = 0  # 记录失败操作的文件数量

    with open(operation_log_path, 'w') as log_file:
        for root, dirs, files in os.walk(directory, topdown=False):
            for name in files:
                if any(name.lower().endswith(ext) for ext in image_extensions):
                    counter += 1
                    source_path = Path(root) / name
                    extension = source_path.suffix
                    new_name = decimal_to_base26(counter)
                    target_path = target_dir / f"{new_name}{extension}"
                    
                    try:
                        shutil.move(str(source_path), str(target_path))
                        print(f"移动并重命名: {source_path} -> {target_path}")
                        successful_files.add(new_name + extension)  # 更新为包含扩展名
                        log_file.write(f"{source_path},{target_path}\n")
                    except Exception as e:
                        print(f"处理文件{source_path}时发生错误: {e}")
                        failed_counter += 1  # 失败操作的文件数量加一

    def clean_non_image_or_unprocessed_files_and_dirs(path, successful_files_set):
        for item in path.iterdir():
            if item.is_file():
                if item.suffix.lower() not in image_extensions or item.name not in successful_files_set:
                    try:
                        if item != operation_log_path:
                            item.unlink()
                            print(f"删除文件: {item}")
                    except Exception as e:
                        print(f"删除文件{item}时发生错误: {e}")
            elif item.is_dir():
                clean_non_image_or_unprocessed_files_and_dirs(item, successful_files_set)
                try:
                    if not any(item.iterdir()) and item != target_dir:
                        item.rmdir()
                        print(f"删除空目录: {item}")
                except Exception as e:
                    print(f"删除空目录{item}时发生错误: {e}")
                    
    clean_non_image_or_unprocessed_files_and_dirs(target_dir, successful_files)

    # 输出成功操作的文件数量和失败操作的文件数量
    total_files = counter
    successful_files_count = len(successful_files)
    failed_files_count = failed_counter
    print(f"处理完成。成功操作文件数量: {successful_files_count}，失败操作文件数量: {failed_files_count}，总共操作文件数量: {total_files}")

def reverse_operations(directory):
    """
    逆向操作，根据操作日志文件中的记录还原文件至原始位置。
    修正：正确处理源路径和目标路径，确保文件从新位置移回到原始位置。
    """
    operation_log_path = Path(directory) / "operation_log.txt"
    if not operation_log_path.exists():
        print("操作日志文件不存在，无法进行逆向操作。")
        return
    
    with open(operation_log_path, 'r') as log_file:
        for line in log_file:
            source_path_str, target_path_str = line.strip().split(',')
            source_path = Path(source_path_str)  # 原始路径
            target_path = Path(target_path_str)     # 当前路径（正向操作后的位置）
            
            try:
                # 确保源路径的父目录存在，因为我们要将文件移回那里
                source_path.parent.mkdir(parents=True, exist_ok=True)
                shutil.move(str(target_path), str(source_path))  # 从当前路径移回原始路径
                print(f"逆向操作完成: {target_path} -> {source_path}")
            except FileNotFoundError:
                print(f"文件{target_path}不存在，跳过逆向操作。")
            except Exception as e:
                print(f"逆向操作时发生错误: {e}")

if __name__ == "__main__":
    target_directory = "/Volumes/192.168.1.173/pic/jjw"
    print("开始处理...")
    move_and_rename_images(target_directory)
    # 可以手动调用逆向操作，但需谨慎处理
    # reverse_operations(target_directory)

开始处理...
移动并重命名: /Volumes/192.168.1.173/pic/jjw/a.jpeg -> /Volumes/192.168.1.173/pic/jjw/a.jpeg
移动并重命名: /Volumes/192.168.1.173/pic/jjw/b.jpg -> /Volumes/192.168.1.173/pic/jjw/b.jpg
移动并重命名: /Volumes/192.168.1.173/pic/jjw/c.jpg -> /Volumes/192.168.1.173/pic/jjw/c.jpg
移动并重命名: /Volumes/192.168.1.173/pic/jjw/d.jpeg -> /Volumes/192.168.1.173/pic/jjw/d.jpeg
移动并重命名: /Volumes/192.168.1.173/pic/jjw/e.jpg -> /Volumes/192.168.1.173/pic/jjw/e.jpg
移动并重命名: /Volumes/192.168.1.173/pic/jjw/f.jpeg -> /Volumes/192.168.1.173/pic/jjw/f.jpeg
移动并重命名: /Volumes/192.168.1.173/pic/jjw/g.jpg -> /Volumes/192.168.1.173/pic/jjw/g.jpg
移动并重命名: /Volumes/192.168.1.173/pic/jjw/h.jpg -> /Volumes/192.168.1.173/pic/jjw/h.jpg
移动并重命名: /Volumes/192.168.1.173/pic/jjw/i.jpg -> /Volumes/192.168.1.173/pic/jjw/i.jpg
移动并重命名: /Volumes/192.168.1.173/pic/jjw/j.jpg -> /Volumes/192.168.1.173/pic/jjw/j.jpg
移动并重命名: /Volumes/192.168.1.173/pic/jjw/k.jpg -> /Volumes/192.168.1.173/pic/jjw/k.jpg
移动并重命名: /Volumes/192.168.1.173/pic/jjw/l.jpg -> /Vo

KeyboardInterrupt: 

In [22]:
def custom_decimal_encoding(n):
    """
    将十进制数字转换为自定义编码，只使用指定的字符进行编码。

    参数:
    n (int): 待转换的十进制数字

    返回:
    str: 自定义编码结果
    """
    # 可使用的字符，排除了容易混淆的字符
    valid_chars = "2346789abcdefghijkmnpqrtuvwxyz"  # 剔除了容易混淆的字符：15os
    valid_chars_length = len(valid_chars)
    print(f"自定义编码字符长度: {valid_chars_length}")
    if n == 0:
        return valid_chars[0]

    result = ""
    while n > 0:
        remainder = n % valid_chars_length
        result = valid_chars[remainder] + result
        n //= valid_chars_length
    return result

# 测试 custom_decimal_encoding 方法
def test_custom_decimal_encoding():
    test_cases = list(range(34))
    # test_cases = [0, 1, 9, 10, 25, 26, 27, 31, 32, 33, 35, 50, 100, 1000, 12345, 987654321]
    for n in test_cases:
        encoded_str = custom_decimal_encoding(n)
        print(f"十进制数字 {n} 转换为自定义编码为: {encoded_str}")

test_custom_decimal_encoding()


自定义编码字符长度: 30
十进制数字 0 转换为自定义编码为: 2
自定义编码字符长度: 30
十进制数字 1 转换为自定义编码为: 3
自定义编码字符长度: 30
十进制数字 2 转换为自定义编码为: 4
自定义编码字符长度: 30
十进制数字 3 转换为自定义编码为: 6
自定义编码字符长度: 30
十进制数字 4 转换为自定义编码为: 7
自定义编码字符长度: 30
十进制数字 5 转换为自定义编码为: 8
自定义编码字符长度: 30
十进制数字 6 转换为自定义编码为: 9
自定义编码字符长度: 30
十进制数字 7 转换为自定义编码为: a
自定义编码字符长度: 30
十进制数字 8 转换为自定义编码为: b
自定义编码字符长度: 30
十进制数字 9 转换为自定义编码为: c
自定义编码字符长度: 30
十进制数字 10 转换为自定义编码为: d
自定义编码字符长度: 30
十进制数字 11 转换为自定义编码为: e
自定义编码字符长度: 30
十进制数字 12 转换为自定义编码为: f
自定义编码字符长度: 30
十进制数字 13 转换为自定义编码为: g
自定义编码字符长度: 30
十进制数字 14 转换为自定义编码为: h
自定义编码字符长度: 30
十进制数字 15 转换为自定义编码为: i
自定义编码字符长度: 30
十进制数字 16 转换为自定义编码为: j
自定义编码字符长度: 30
十进制数字 17 转换为自定义编码为: k
自定义编码字符长度: 30
十进制数字 18 转换为自定义编码为: m
自定义编码字符长度: 30
十进制数字 19 转换为自定义编码为: n
自定义编码字符长度: 30
十进制数字 20 转换为自定义编码为: p
自定义编码字符长度: 30
十进制数字 21 转换为自定义编码为: q
自定义编码字符长度: 30
十进制数字 22 转换为自定义编码为: r
自定义编码字符长度: 30
十进制数字 23 转换为自定义编码为: t
自定义编码字符长度: 30
十进制数字 24 转换为自定义编码为: u
自定义编码字符长度: 30
十进制数字 25 转换为自定义编码为: v
自定义编码字符长度: 30
十进制数字 26 转换为自定义编码为: w
自定义编码字符长度: 30
十进制数字 27 转换为自定义编码为: x
自定

In [28]:
def decimal_to_mixed_base(n):
    """将十进制数字转换为自定义基数的字母数字混合表示，基数由allowed_chars长度决定"""
    allowed_chars = "0123456789ABCDEFGHJKLMNPQRSTUVWXYZabcdefghjkmnpqrstuvwxyz"
    base = len(allowed_chars)
    result = ""
    
    # 处理负数
    is_negative = n < 0
    n = abs(n)
    
    while n > 0:
        n, remainder = divmod(n, base)
        result = allowed_chars[remainder] + result
    
    # 对于负数，在结果前添加负号
    if is_negative:
        result = '-' + result
    
    # 特殊处理n为0的情况
    elif not result:
        result = allowed_chars[0]
        
    return result

def test_decimal_to_mixed_base():
    print("测试正常数值:")
    print(f"0 -> {decimal_to_mixed_base(0)}")  # 应当输出 "0"
    print(f"21 -> {decimal_to_mixed_base(21)}")  # 应当输出 "T"
    print(f"22 -> {decimal_to_mixed_base(22)}")  # 应当输出 "10"
    print(f"45 -> {decimal_to_mixed_base(45)}")  # 应当输出 "21"
    
    print("\n测试大数值:")
    print(f"1000 -> {decimal_to_mixed_base(1000)}")  # 应当输出 "1F6"
    print(f"2222 -> {decimal_to_mixed_base(2222)}")  # 用于检验边界处理，实际输出会是自定义基数表示
    
    print("\n测试边缘情况:")
    print(f"-1 -> {decimal_to_mixed_base(-1)}")  # 应当输出 "-1"
    
    print("测试完成。")

# 运行测试
test_decimal_to_mixed_base()

测试正常数值:
0 -> 0
21 -> M
22 -> N
45 -> n

测试大数值:
1000 -> HX
2222 -> ez

测试边缘情况:
-1 -> -1
测试完成。


In [39]:
def decimal_to_custom_base(n):
    """将十进制数字转换为25进制，使用非容易混淆的字符编码"""
    characters = 'acdefhjkmnpqrtuvwxyz23478'
    base = len(characters)
    if n == 0:
        return characters[0]  # 将0映射到第一个字符'a'
    result = ""
    while n > 0:
        n, remainder = divmod(n, base)
        result = characters[remainder] + result
    return result

# 测试函数
def test_decimal_to_custom_base():
    test_cases = [
        (0, "a"),
        (1, "c"),
        (25, "ca"),
        (50, "ec"),
        (75, "kc"),
        (100, "rc"),
        (500, "trc"),
        (1000, "euc"),
        (1234, "mqc")
    ]

    for number, expected in test_cases:
        result = decimal_to_custom_base(number)
        assert result == expected, f"对于输入 {number}, 期望得到 '{expected}', 但实际得到 '{result}'"
        print(f"输入: {number}, 输出: {result}, 预期: {expected}")

    print("所有测试用例均通过")

# 运行测试函数
test_decimal_to_custom_base()


输入: 0, 输出: a, 预期: a
输入: 1, 输出: c, 预期: c
输入: 25, 输出: ca, 预期: ca


AssertionError: 对于输入 50, 期望得到 'ec', 但实际得到 'da'